### dynamically grab features from rectangle & match BETTER

In [1]:
import numpy as np
import cv2

In [8]:
ret, frame = cv2.VideoCapture(0).read()
frame_height, frame_width = frame.shape[:2]

print(frame_height, frame_width)

480 640


In [9]:
settings = { 
    "frame_height": frame_height,
    "frame_width": frame_width
}

In [10]:
def blank_overlay(height, width):
    return np.zeros((height, width, 3), dtype=np.uint8)

def rectangle_creation(event, x, y, flags, param):
    global temp_point, rectangles
    # if the left mouse button was clicked, record the starting
    if event == cv2.EVENT_LBUTTONDOWN:
        temp_point = (x, y)
    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        if(temp_point):
            rectangles = [[temp_point, (x, y)]] # we only use one for now
            recalculate_region(rectangles[0])    
        temp_point = None
        
def recalculate_region(r):
    global rectangles, overlay, feature_region, frame, kp_fr, des_fr, frame_height, frame_width
    
    print(r[0], r[1])
    
    overlay = blank_overlay(frame_height, frame_width)
    cv2.rectangle(overlay, r[0], r[1], (0, 255, 0), 2)
    
    feature_region = frame[r[0][1]:r[1][1], r[0][0]:r[1][0]]
    
    kp_fr, des_fr = orb.detectAndCompute(feature_region, None)
    
    feature_region = cv2.drawKeypoints(frame[r[0][1]:r[1][1], r[0][0]:r[1][0]], kp_fr, None, color=(0,255,0),  flags=cv2.DrawMatchesFlags_DEFAULT)

    

In [11]:

# now let's initialize the list of reference point
temp_point = []
rectangles = [] 
feature_region = np.zeros((1, 1, 3), dtype=np.uint8)

kp_fr = None
des_fr = None

overlay = blank_overlay(frame_height, frame_width)

cv2.namedWindow("image")
cv2.setMouseCallback("image", rectangle_creation)

cap = cv2.VideoCapture(0)

alpha = 0.5

# Initiate ORB detector
orb = cv2.ORB_create(nfeatures=2000)

# create BFMatcher object
bf = cv2.BFMatcher()



In [12]:
cap = cv2.VideoCapture(0)
### LOOOP

while True:
    ret, frame = cap.read()

    kp, des = orb.detectAndCompute(frame,None)

    if(np.any(des_fr)):
        # Match descriptors.
        matches = bf.knnMatch(des_fr, des, k=2)

        # store all the good matches as per Lowe's ratio test.
        good = []
        for m,n in matches:
            if m.distance < 0.75 * n.distance:
                good.append([m])
            
                
        # Draw first 10 matches.
        frame = cv2.drawMatchesKnn(feature_region,kp_fr,frame,kp,good,None, flags=2)

    
    # picture in picture
#    frame[50:50+feature_region.shape[0], 50:50+feature_region.shape[1]] = feature_region    
#    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    
    # display the image and wait for a keypress
    cv2.imshow("image", frame)
    
    key = cv2.waitKey(1) & 0xFF
    # press 'r' to reset the window
    if key == ord("r"):
        overlay = new_overlay()
    # if the 'c' key is pressed, break from the loop
    if key == ord("c"):
        break
        
# close all open windows
cap.release()
cv2.destroyAllWindows() 


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\highgui\src\window.cpp:356: error: (-215) size.width>0 && size.height>0 in function cv::imshow
